In [1]:
import os
import urllib
import cv2
import numpy as np
from pixellib.semantic import semantic_segmentation
from matplotlib import pyplot as plt

In [2]:
img_path = os.getenv('HOME')+'/aiffel/human_segmentation/images/my_second_image.png'  
img_orig = cv2.imread(img_path) 

print(img_orig.shape)

plt.imshow(cv2.cvtColor(img_orig, cv2.COLOR_BGR2RGB))
plt.show()

AttributeError: 'NoneType' object has no attribute 'shape'

In [ ]:
model_dir = os.getenv('HOME')+'/aiffel/human_segmentation/models' 
model_file = os.path.join(model_dir, 'deeplabv3_xception_tf_dim_ordering_tf_kernels.h5') 
model_url = 'https://github.com/ayoolaolafenwa/PixelLib/releases/download/1.1/deeplabv3_xception_tf_dim_ordering_tf_kernels.h5' 

urllib.request.urlretrieve(model_url, model_file) # urllib 패키지 내에 있는 request 모듈의 urlretrieve 함수를 이용해서 model_url에 있는 파일을 다운로드 해서 model_file 파일명으로 저장

In [ ]:
model = semantic_segmentation() #PixelLib 라이브러리 에서 가져온 클래스를 가져와서 semantic segmentation을 수행하는 클래스 인스턴스를 만듬
model.load_pascalvoc_model(model_file) # pascal voc에 대해 훈련된 예외 모델(model_file)을 로드하는 함수를 호출

In [ ]:
segvalues, output = model.segmentAsPascalvoc(img_path) 
# segmentAsPascalvoc()함 수 를 호출 하여 입력된 이미지를 분할, 분할 출력의 배열을 가져옴, 분할 은 pacalvoc 데이터로 학습된 모델을 이용

In [ ]:
#pascalvoc 데이터의 라벨종류
LABEL_NAMES = [
    'background', 'aeroplane', 'bicycle', 'bird', 'boat', 'bottle', 'bus',
    'car', 'cat', 'chair', 'cow', 'diningtable', 'dog', 'horse', 'motorbike',
    'person', 'pottedplant', 'sheep', 'sofa', 'train', 'tv'
]
len(LABEL_NAMES)

In [ ]:
#segmentAsPascalvoc() 함수 를 호출하여 입력된 이미지를 분할한 뒤 나온 결과값 중 output을 matplotlib을 이용해 출력
plt.imshow(output)
plt.show()

In [ ]:
segvalues # segmentAsPascalvoc() 함수를 호출하여 입력된 이미지를 분할한 뒤 나온 결과값 중 배열값을 출력

In [ ]:
#segvalues에 있는 class_ids를 담겨있는 값을 통해 pacalvoc에 담겨있는 라벨을 출력
for class_id in segvalues['class_ids']:
    print(LABEL_NAMES[class_id])

In [ ]:
#컬러맵 만들기 
colormap = np.zeros((256, 3), dtype = int)
ind = np.arange(256, dtype=int)

for shift in reversed(range(8)):
    for channel in range(3):
        colormap[:, channel] |= ((ind >> channel) & 1) << shift
    ind >>= 3

colormap[:20] #생성한 20개의 컬러맵 출력

In [ ]:
colormap[15] #컬러맵 15에 해당하는 배열 출력 (pacalvoc에 LABEL_NAMES 15번째인 사람)

In [ ]:
seg_color = (128,128,192) # 색상순서 변경 - colormap의 배열은 RGB 순이며 output의 배열은 BGR 순서로 채널 배치가 되어 있어서

In [ ]:
# output의 픽셀 별로 색상이 seg_color와 같다면 1(True), 다르다면 0(False)이 됩니다
# seg_color 값이 person을 값이 므로 사람이 있는 위치를 제외하고는 gray로 출력
# cmap 값을 변경하면 다른 색상으로 확인이 가능함
seg_map = np.all(output==seg_color, axis=-1) 
print(seg_map.shape) 
plt.imshow(seg_map, cmap='gray')
plt.show()

In [ ]:
# 문제점 해결 방안
# Semantic segmentation에서 피사계 심도를 표현하는 아웃포커싱 효과를 구현하는 데는 한계가 있는데, 특히 경계가 불명확한 영역에서 그렇습니다. 
# 깊이 예측(Depth Estimation)과 함께 semantic segmentation을 결합하여 응용하는 방식으로 예시를 들어 보겠습니다.

In [ ]:
import torch
import cv2
import numpy as np
import matplotlib.pyplot as plt
from torchvision import transforms
from PIL import Image

# MiDaS 모델 불러오기
midas = torch.hub.load("intel-isl/MiDaS", "MiDaS")
midas_transforms = torch.hub.load("intel-isl/MiDaS", "transforms")

# DeepLabV3 모델 불러오기
deeplab = torch.hub.load('pytorch/vision:v0.10.0', 'deeplabv3_resnet101', pretrained=True)
deeplab.eval()

# 이미지 불러오기
img = cv2.imread("my_second_image.jpg")
img_rgb = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)

# 깊이 예측
input_batch = midas_transforms(img_rgb).unsqueeze(0)
with torch.no_grad():
    prediction = midas(input_batch)
    prediction = torch.nn.functional.interpolate(
        prediction.unsqueeze(1),
        size=img_rgb.shape[:2],
        mode="bicubic",
        align_corners=False,
    ).squeeze()
depth_map = prediction.cpu().numpy()

# Semantic segmentation 예측
preprocess = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize(520),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225]),
])
input_tensor = preprocess(img_rgb)
input_batch = input_tensor.unsqueeze(0)

with torch.no_grad():
    output = deeplab(input_batch)['out'][0]
output_predictions = output.argmax(0).byte().cpu().numpy()

# 깊이 맵과 세그멘테이션 결과 결합
def combine_depth_and_segmentation(depth, segmentation, threshold=0.5):
    combined = np.zeros_like(segmentation)
    for i in range(segmentation.max() + 1):
        mask = (segmentation == i)
        depth_values = depth[mask]
        if depth_values.mean() > threshold:
            combined[mask] = i
    return combined

combined_result = combine_depth_and_segmentation(depth_map, output_predictions)

# 결과 시각화
fig, ax = plt.subplots(1, 3, figsize=(15, 5))
ax[0].imshow(depth_map, cmap='plasma')
ax[0].set_title('Depth Map')
ax[1].imshow(output_predictions)
ax[1].set_title('Segmentation')
ax[2].imshow(combined_result)
ax[2].set_title('Combined Result')
plt.show()
